### 모델 학습 코드, 배포 모델은 hate_model.py 확인

In [1]:
import pandas as pd
train = pd.read_csv('hate_train_spaced.csv')
test = pd.read_csv('hate_test_spaced.csv')
train = pd.concat([train, test], ignore_index=True)
train.head(5)

,comments,hate,label
0,현재 호텔 주인 심정 아 난 마른 하늘에 날 벼락 맞고 호텔 망하게 생겼는데 누군 ...,hate,1
1,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미 처...,none,0
2,못된 넘들남의 고통을 즐겼던 넘들이 젠 마땅한 처벌을 받아야지 그래야 공정한 사회 ...,hate,1
3,화 어설 펐는데 화 지나서부터는 갈수록 너무 재밌던 데,none,0
4,사람 얼굴 손톱으로 긁은 것은 인격 살해이고 동영상이 몰카냐 메걸리안 들 생각이 없노,hate,1


In [2]:
train['label'].value_counts()

1    4721
0    3646
Name: label, dtype: int64

In [3]:
from transformers import ElectraTokenizer

train_X = train['comments'].tolist()

tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')
train_X = tokenizer(train_X, truncation=True, padding=True)

In [4]:
import numpy as np
train_toks = np.asarray(train_X['input_ids'])
train_masks = np.asarray(train_X['attention_mask'])

train_y = np.asarray(train['label'])

print(train_toks.shape)
print(train_masks.shape)
print(train_y.shape)

emb_len = train_toks.shape[-1]

(8367, 76)
(8367, 76)
(8367,)


In [5]:
from transformers import logging, TFElectraModel
logging.set_verbosity_error()
import tensorflow as tf

electra_model = TFElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator', from_pt=True)

input_toks = tf.keras.layers.Input(shape=(emb_len,), name='toks', dtype='int32')
input_masks = tf.keras.layers.Input(shape=(emb_len,), name='masks', dtype='int32')

electra_output = electra_model(input_toks, attention_mask=input_masks).last_hidden_state

x = tf.keras.layers.GlobalAveragePooling1D()(electra_output)
# x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(32, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.2)(x)
y = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=[input_toks, input_masks], outputs=y)
adam = tf.keras.optimizers.Adam(learning_rate=4e-5)

model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
toks (InputLayer)               [(None, 76)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 76)]         0                                            
__________________________________________________________________________________________________
tf_electra_model (TFElectraMode TFBaseModelOutput(la 112330752   toks[0][0]                       
                                                                 masks[0][0]                      
__________________________________________________________________________________________________
global_average_pooling1d (Globa (None, 768)          0           tf_electra_model[0][0]

In [6]:
history = model.fit(x={'toks':train_toks, 'masks':train_masks}, y=train_y, epochs=5, validation_split=0.2)

Epoch 1/5
210/210 [==============================] - 104s 495ms/step - loss: 0.5848 - accuracy: 0.6906 - val_loss: 0.4542 - val_accuracy: 0.7724
Epoch 2/5
210/210 [==============================] - 107s 508ms/step - loss: 0.4261 - accuracy: 0.8114 - val_loss: 0.4755 - val_accuracy: 0.7670
Epoch 3/5
106/210 [==============>...............] - ETA: 46s - loss: 0.2854 - accuracy: 0.8847

KeyboardInterrupt: 

In [6]:
model.fit(x={'toks':train_toks, 'masks':train_masks}, y=train_y, epochs=2)

Epoch 1/2
262/262 [==============================] - 112s 429ms/step - loss: 0.5172 - accuracy: 0.7390
Epoch 2/2
262/262 [==============================] - 127s 484ms/step - loss: 0.3648 - accuracy: 0.8338


In [7]:
test_X = tokenizer(['아니 진짜 이런걸 왜 모르지 고등학생도 알겠다 대학생 맞냐'], truncation=True, padding='max_length', max_length=emb_len)
test_toks = np.asarray(test_X['input_ids'])
test_masks = np.asarray(test_X['attention_mask'])

print(test_toks.shape)
print(test_masks.shape)

(1, 76)
(1, 76)


In [8]:
model.predict(x={'toks':test_toks, 'masks':test_masks})

array([[0.13739891, 0.86260116]], dtype=float32)

In [10]:
model.save_weights('hate_weights/')

In [13]:
test_texts = ['아니 이런걸 왜 모르지 이런건 고등학생도 알겠다 대학생 맞냐', 
                '신공학관 1층에도 있고 학생회관 2층에도 있음',
                '니가 먼저 직접 알아보고 여기다가 질문하라고',
                '새천년관 가려면 정문보다는 후문이 더 빠르지',
                '제발 좆 같은 소리 좀 하지마',
                '제발 그만 해']

In [14]:
test_X = tokenizer(test_texts, truncation=True, padding='max_length', max_length=emb_len)
test_toks = np.asarray(test_X['input_ids'])
test_masks = np.asarray(test_X['attention_mask'])

pred = model.predict(x={'toks':test_toks, 'masks':test_masks})

print(pred)

THRESH = 0.7
for i, entry in enumerate(test_texts):
    if pred[i][0] >= THRESH:
        print(entry, '-> 악플이 아닙니다')
    else:
        print(entry, '-> 악플입니다')

[[0.12981658 0.8701834 ]
 [0.96772957 0.03227039]
 [0.1010987  0.8989013 ]
 [0.9101211  0.08987894]
 [0.06932552 0.93067443]
 [0.58639854 0.41360143]]
아니 이런걸 왜 모르지 이런건 고등학생도 알겠다 대학생 맞냐 -> 악플입니다
신공학관 1층에도 있고 학생회관 2층에도 있음 -> 악플이 아닙니다
니가 먼저 직접 알아보고 여기다가 질문하라고 -> 악플입니다
새천년관 가려면 정문보다는 후문이 더 빠르지 -> 악플이 아닙니다
제발 좆 같은 소리 좀 하지마 -> 악플입니다
제발 그만 해 -> 악플입니다
